In [ ]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

In [ ]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

## https://developer.hashicorp.com/vault/docs/auth/jwt/oidc-providers/kubernetes

In [ ]:
%%bash
kubectl create clusterrolebinding oidc-reviewer  \
   --clusterrole=system:service-account-issuer-discovery \
   --group=system:unauthenticated


In [ ]:
%%bash
ISSUER="$(kubectl get --raw /.well-known/openid-configuration | jq -r '.issuer')"
echo $ISSUER

In [ ]:
%%bash
kubectl exec vault-0 -n vault -- vault login hvs.RD8vp71RDNbGBTFcKJuD1sFV
kubectl exec vault-0 -n vault -- vault auth enable jwt
kubectl exec vault-0 -n vault -- vault write auth/jwt/config \
   oidc_discovery_url=https://kubernetes.default.svc.cluster.local \
   # Not required when using EKS
   oidc_discovery_ca_pem=@/var/run/secrets/kubernetes.io/serviceaccount/ca.crt

In [ ]:

! kubectl get --raw "$(kubectl get --raw /.well-known/openid-configuration | jq -r '.jwks_uri' | sed -r 's/.*\.[^/]+(.*)/\1/')" | jq -r
! pip3 install pycryptodome

In [ ]:
import base64
import binascii
from Crypto.PublicKey import RSA

jwks_key = {
  "keys": [
    {
      "use": "sig",
      "kty": "RSA",
      "kid": "tk1afHmOlH6WXDmueZ7Qep6flST1vGLauAOYspfPO6o",
      "alg": "RS256",
      "n": "zBNt_YeRx4WwYWtNwoDhu08O1rPabTZCE7yeEMOe4HCiJb6QSOYocR2NPE3riag878mMjk5NqWvJ6mOhm3z6jnGRUSSc0ZKt5Jld0a4ZA0Ju361_cDQVEy88Tw4a_0kiS8-tmKT2LimPbKMbQVNZM5rgyiG-CFt0hm6edP5P_tmQN93P_m7jBq9G-4NWHGm0G1Y4oMEsa4h5dVeMX4f9-9zxLdlDVcT3JKdA4xH_0zPXfZK0BXi642sSqIV8fyQXXNrhfIwgGrPEFj4_PEJ3XdDy_FQRVGQDLAs8kWHSJLtkmX4yebrPbMgBRxWUSCg-2mtiX1ZOvU96k9YRVKLKjw",
      "e": "AQAB"
    }
  ]
}

# Select the specific key from the JWKS
selected_key = jwks_key['keys'][0]

# Decode the base64 values of 'n' and 'e' to bytes
n_bytes = base64.urlsafe_b64decode(selected_key['n'] + '==')
e_bytes = base64.urlsafe_b64decode(selected_key['e'] + '==')

# Construct an RSA key
rsa_key = RSA.construct((int(binascii.hexlify(n_bytes), 16), int(binascii.hexlify(e_bytes), 16)))

# Convert the RSA key to PEM format
pem_key = rsa_key.exportKey(format='PEM')

print(pem_key.decode('utf-8'))


In [ ]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

In [ ]:
%env VAULT_ADDR=https://127.0.0.1:8200
%env VAULT_TOKEN=hvs.RD8vp71RDNbGBTFcKJuD1sFV
%env VAULT_CACERT=/tmp/vault/vault.ca

In [ ]:
%%bash
# Role associated to default service account in the app namespace
vault write auth/kubernetes/role/role1 \
    bound_service_account_names=default \
    bound_service_account_namespaces=vault \
    policies=devk8s \
    ttl=24h


In [ ]:
%%bash
# Role associated to default service account in the app namespace
vault write auth/kubernetes/role/role2 \
    bound_service_account_names=default \
    bound_service_account_namespaces=default \
    policies=devk8s \
    ttl=24h

In [ ]:
%%bash
vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read"]
}
EOF


In [ ]:
%%bash
vault secrets enable -path=kvv2 kv-v2

In [ ]:
%%bash
vault kv put kvv2/webapp/config username="static-user" password="static-password"

### Instalando el VSO usando helm

In [ ]:
%%bash
helm repo add hashicorp https://helm.releases.hashicorp.com
helm repo update hashicorp
helm install --namespace $VAULT_K8S_NAMESPACE vault-secrets-operator hashicorp/vault-secrets-operator


In [ ]:
%%bash
## VaultConnection Custom Resource
# https://developer.hashicorp.com/vault/docs/platform/k8s/vso/sources/vault
cat > vso_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultConnection
metadata:
  namespace: vault
  name: example
spec:
  address: https://vault.vault.svc.cluster.local:8200
  skipTLSVerify: true

EOF
kubectl apply -f vso_crd.yaml
echo '---'
kubectl describe VaultConnection example -n vault


In [ ]:
%%bash
# https://developer.hashicorp.com/vault/docs/platform/k8s/vso/api-reference#vaultauthconfigjw

cat > vaultauth_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultAuth
metadata:
  namespace: vault
  name: example
spec:
  # configured in its own Kubernetes namespace.
  vaultConnectionRef: example
  # Method to use when authenticating to Vault.
  method: jwt
  # Mount to use when authenticating to auth method.
  mount: jwt
  # allowedNamespaces: []{"*"}
  # Kubernetes specific auth configuration, requires that the Method be set to kubernetes.
  jwt:
    # role to use when authenticating to Vault
    role: my-role3
    serviceAccount: test
    audiences: ["https://kubernetes.default.svc.cluster.local"]

EOF

kubectl apply -f vaultauth_crd.yaml
echo '---'
kubectl describe VaultAuth example -n vault

## CRD Static Secret

In [ ]:
%%bash
## Support KVv1 and KVv2
cat > secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  namespace: vault
  name: example2
spec:
  vaultAuthRef: example
  mount: kvv2
  type: kv-v2
  path: webapp/config
  refreshAfter: 60s
  destination:
    create: true
    name: secret
EOF
kubectl apply -f secret.yaml
echo '---'
kubectl describe VaultStaticSecret example2 -n vault
echo '---'
sleep 5
kubectl get secret secret -n vault -o yaml


In [ ]:
%%bash
## Support KVv1 and KVv2
export NAMESPACE=vault111
cat > secret_${NAMESPACE}.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  namespace: $NAMESPACE
  name: example2
spec:
  vaultAuthRef: example
  mount: kvv2
  type: kv-v2
  path: webapp/config
  refreshAfter: 60s
  destination:
    create: true
    name: secret
EOF
kubectl apply -f secret_${NAMESPACE}.yaml
echo '---'
kubectl describe VaultStaticSecret example2 -n $NAMESPACE
echo '---'
sleep 5
kubectl get secret secret -n $NAMESPACE -o yaml


In [ ]:
%%bash
echo "namespace: Vault"
kubectl get secrets -n vault

In [ ]:
%%bash
cat > mypod.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypod
  namespace: vault
spec:
  containers:
  - name: mypod
    image: nginx
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: static-secret1
      optional: true
EOF
kubectl delete -f mypod.yaml
sleep 10

# https://developer.hashicorp.com/vault/tutorials/kubernetes/vault-secrets-operator

In [ ]:
%%bash 
kubectl exec mypod -n vault -- ls /etc/foo
echo ""
echo "------"
kubectl exec mypod -n vault -- cat /etc/foo/_raw